In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import gc
from tqdm import tqdm

TEST_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\test.csv'
ADDITIONAL_TRAIN_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\last_point.csv'
MODELS_DIR = r'C:\Users\e0817820\Desktop\tokka\models'

C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\636159681.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
use_additional_train = False

In [3]:
list_cryptocurrencies = ['AVAX', 'ADA', 'SOL', 'BNB', 'TRX', 'DOGE', 'LINK', 'XRP', 'BTC', 'ETH']

df_train = pd.read_csv(ADDITIONAL_TRAIN_CSV)
df_test = pd.read_csv(TEST_CSV)
lower_limit_test = df_test['timestamp'].min()
upper_limit_train = df_train['timestamp'].max()
lower_limit_test_datetime = pd.to_datetime(df_test['timestamp'].min(), unit='ms')
upper_limit_train_datetime = pd.to_datetime(df_train['timestamp'].max(), unit='ms')
# df_train.drop(columns=['Unnamed: 0'])

In [4]:
print(df_test.shape)

(1226990, 12)


In [5]:
df_combined = pd.concat([df_train, df_test])
df_combined['timestamp'] = pd.to_datetime(df_combined['timestamp'], unit='ms')
all_timestamps = pd.date_range(start=upper_limit_train_datetime, end=lower_limit_test_datetime, freq='T')  # Adjust the freq='T' as necessary

expanded_df = pd.DataFrame()
for symbol in list_cryptocurrencies:
    temp_df = pd.DataFrame({'timestamp': all_timestamps})
    temp_df['symbol'] = symbol 
    temp_df = pd.merge(temp_df, df_combined[df_combined['symbol'] == symbol], on='timestamp', how='left', suffixes=('', '_drop'))
    temp_df.drop([col for col in temp_df if col.endswith('_drop')], axis=1, inplace=True)
    expanded_df = pd.concat([expanded_df, temp_df])

expanded_df = expanded_df.sort_values(by=['timestamp', 'symbol'], ascending=[True, True])
expanded_df['timestamp'] = (expanded_df['timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

df_filled = pd.concat([expanded_df, df_test])
df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))
df_filled.reset_index(drop=True, inplace=True)
df_filled.head()
del df_combined

df_filled.rename(columns={'index': 'timestamp'}, inplace=True)
df_filled.drop(columns=['id'], inplace=True)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\3585968195.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  all_timestamps = pd.date_range(start=upper_limit_train_datetime, end=lower_limit_test_datetime, freq='T')  # Adjust the freq='T' as necessary
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\3585968195.py:17: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\3585968195.py:17: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))
C:\Users

In [6]:
print(df_filled.shape)

(1233320, 12)


In [7]:
# df_filled.to_csv(r'C:\Users\e0817820\Desktop\tokka\data\raw\test_filled.csv', index=False)

In [8]:
df_filled.head()

,timestamp,symbol,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_volume,taker_sell_volume,log_return
0,1691345340000,ADA,0.292300,0.292400,0.292300,0.292400,3343.500000,977.553110,34.000000,2151.300000,629.040120,0.0
1,1691345400000,ADA,0.292303,0.292403,0.292303,0.292403,3353.617405,980.541681,33.963608,2156.203006,630.490373,0.0
2,1691345460000,ADA,0.292306,0.292406,0.292306,0.292406,3363.734810,983.530252,33.927215,2161.106013,631.940626,0.0
3,1691345520000,ADA,0.292309,0.292409,0.292309,0.292409,3373.852215,986.518824,33.890823,2166.009019,633.390879,0.0
4,1691345580000,ADA,0.292313,0.292413,0.292313,0.292412,3383.969620,989.507395,33.854430,2170.912025,634.841132,0.0


In [9]:
combined_df1 = pd.DataFrame()
combined_df1[df_filled.columns] = 0
for id in range(10):
    combined_df1 = combined_df1.merge(df_filled.loc[df_filled["symbol"] == list_cryptocurrencies[id], ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'log_return']].copy(), on="timestamp", how='outer', suffixes=['', "_"+list_cryptocurrencies[id]])
print(combined_df1.isna().sum())
combined_df1 = combined_df1.drop(df_filled.columns.drop("timestamp"), axis=1)
combined_df1 = combined_df1.sort_values('timestamp', ascending=True)
combined_df1 = combined_df1.fillna(method='ffill')
combined_df1.drop_duplicates(subset=['timestamp'], inplace=True)

combined_df1 = combined_df1[combined_df1['timestamp'] > lower_limit_test - 400*60*1000]
display(combined_df1.head())

timestamp              0
symbol            124354
open              124354
high              124354
low               124354
                   ...  
high_ETH               0
low_ETH                0
close_ETH              0
volume_ETH             0
log_return_ETH         0
Length: 72, dtype: int64


C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\2459315739.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df1 = combined_df1.fillna(method='ffill')


,timestamp,open_AVAX,high_AVAX,low_AVAX,close_AVAX,volume_AVAX,log_return_AVAX,open_ADA,high_ADA,low_ADA,...,low_BTC,close_BTC,volume_BTC,log_return_BTC,open_ETH,high_ETH,low_ETH,close_ETH,volume_ETH,log_return_ETH
233,1691359320000,12.637927,12.637927,12.637927,12.637927,341.095127,-0.000795,0.293037,0.293137,0.293037,...,29131.831044,29131.841044,3.901821,-0.000027,1836.244019,1836.244019,1836.230332,1836.240332,13.614306,0.000464
234,1691359380000,12.638133,12.638133,12.638133,12.638133,342.537595,-0.000795,0.293041,0.293141,0.293041,...,29131.967658,29131.977658,3.902867,-0.000027,1836.256139,1836.256139,1836.242437,1836.252437,13.627476,0.000464
235,1691359440000,12.638339,12.638339,12.638339,12.638339,343.980063,-0.000795,0.293044,0.293144,0.293044,...,29132.104272,29132.114272,3.903912,-0.000027,1836.268259,1836.268259,1836.254541,1836.264541,13.640647,0.000464
236,1691359500000,12.638544,12.638544,12.638544,12.638544,345.422532,-0.000795,0.293047,0.293147,0.293047,...,29132.240886,29132.250886,3.904957,-0.000027,1836.280380,1836.280380,1836.266646,1836.276646,13.653817,0.000464
237,1691359560000,12.638750,12.638750,12.638750,12.638750,346.865000,-0.000795,0.293050,0.293150,0.293050,...,29132.377500,29132.387500,3.906003,-0.000027,1836.292500,1836.292500,1836.278750,1836.288750,13.666987,0.000464


In [10]:
combined_df2 = pd.DataFrame()
combined_df2[df_filled.columns] = 0
for id in range(10):
    combined_df2 = combined_df2.merge(df_filled.loc[df_filled["symbol"] == list_cryptocurrencies[id], ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'log_return']].copy(), on="timestamp", how='outer', suffixes=['', "_"+list_cryptocurrencies[id]])
combined_df2 = combined_df2.drop(df_filled.columns.drop("timestamp"), axis=1)
combined_df2 = combined_df2.sort_values('timestamp', ascending=True)
combined_df2 = combined_df2.fillna(method='ffill')
combined_df2.drop_duplicates(subset=['timestamp'], inplace=True)

combined_df2 = combined_df2[combined_df2['timestamp'] > lower_limit_test - 400*60*1000]
display(combined_df2.head())

C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\4248047735.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df2 = combined_df2.fillna(method='ffill')


,timestamp,open_AVAX,high_AVAX,low_AVAX,close_AVAX,volume_AVAX,log_return_AVAX,open_ADA,high_ADA,low_ADA,...,low_BTC,close_BTC,volume_BTC,log_return_BTC,open_ETH,high_ETH,low_ETH,close_ETH,volume_ETH,log_return_ETH
233,1691359320000,12.637927,12.637927,12.637927,12.637927,341.095127,-0.000795,0.293037,0.293137,0.293037,...,29131.831044,29131.841044,3.901821,-0.000027,1836.244019,1836.244019,1836.230332,1836.240332,13.614306,0.000464
234,1691359380000,12.638133,12.638133,12.638133,12.638133,342.537595,-0.000795,0.293041,0.293141,0.293041,...,29131.967658,29131.977658,3.902867,-0.000027,1836.256139,1836.256139,1836.242437,1836.252437,13.627476,0.000464
235,1691359440000,12.638339,12.638339,12.638339,12.638339,343.980063,-0.000795,0.293044,0.293144,0.293044,...,29132.104272,29132.114272,3.903912,-0.000027,1836.268259,1836.268259,1836.254541,1836.264541,13.640647,0.000464
236,1691359500000,12.638544,12.638544,12.638544,12.638544,345.422532,-0.000795,0.293047,0.293147,0.293047,...,29132.240886,29132.250886,3.904957,-0.000027,1836.280380,1836.280380,1836.266646,1836.276646,13.653817,0.000464
237,1691359560000,12.638750,12.638750,12.638750,12.638750,346.865000,-0.000795,0.293050,0.293150,0.293050,...,29132.377500,29132.387500,3.906003,-0.000027,1836.292500,1836.292500,1836.278750,1836.288750,13.666987,0.000464


In [11]:
print(combined_df1.shape)
print(combined_df2.shape)

(123098, 61)
(123098, 61)


In [12]:
def bollinger_mavg(close, window=20):
    """Calculate the middle Bollinger Band."""
    return close.rolling(window=window).mean()

def bollinger_hband(close, window=20, k=2):
    """Calculate the upper Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg + (mstd * k)

def bollinger_lband(close, window=20, k=2):
    """Calculate the lower Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg - (mstd * k)

def bollinger_wband(close, window=20, k=2):
    """Calculate the width of the Bollinger Bands."""
    hband = bollinger_hband(close, window, k)
    lband = bollinger_lband(close, window, k)
    mavg = bollinger_mavg(close, window)
    return ((hband - lband) / mavg) * 100

def exponential_moving_average(close, span=20):
    """Calculate the Exponential Moving Average (EMA)."""
    return close.ewm(span=span, adjust=False).mean()

def relative_strength_index(close, window=14):
    """Calculate the Relative Strength Index (RSI)."""
    delta = close.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi
  
def stochastic_oscillator(high, low, close, window=14):
    """Calculate the Stochastic Oscillator."""
    low_min = low.rolling(window=window).min()
    high_max = high.rolling(window=window).max()
    k = ((close - low_min) / (high_max - low_min)) * 100
    return k

def williams_r(high, low, close, window=14):
    """Calculate Williams %R."""
    high_max = high.rolling(window=window).max()
    low_min = low.rolling(window=window).min()
    r = -100 * ((high_max - close) / (high_max - low_min))
    return r
  
def rate_of_change(close, window=14):
    """Calculate the Rate of Change (ROC)."""
    roc = ((close - close.shift(window)) / close.shift(window)) * 100
    return roc

In [13]:
def delay(x, d):
    """Value of x d days ago"""
    return x.shift(d)

def correlation(x, y, d):
    """Time-series correlation of x and y for the past d days"""
    return x.rolling(window=d).corr(y)

def delta(x, d):
    """Today's value of x minus the value of x d days ago"""
    return x.diff(d)

def ts_max(x, d):
    """Time-series max over the past d days"""
    return x.rolling(window=d).max()

def ts_min(x, d):
    """Time-series min over the past d days"""
    return x.rolling(window=d).min()

In [14]:
def getFeaturesVer1(df, limit):
    lags_normal = [15, 40, 150, 300]
    lags_ema = [15, 50, 160, 300]
    lags_rsi = [15, 50, 160, 300]
    lags_bbands = [15, 50, 160, 300]
    for id in range(10):   
        symbol = list_cryptocurrencies[id] 
        for lag in lags_normal:
            df[f'log_close/mean_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.append(np.convolve( np.array(df[f'close_{symbol}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1)  )
            df[f'log_return_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.array(df[f'close_{symbol}']), lag)  )
    for lag in lags_normal:
        df[f'mean_close/mean_{lag}'] =  np.mean(df.iloc[:,df.columns.str.startswith(f'log_close/mean_{lag}_')], axis=1)
        df[f'mean_log_returns_{lag}'] = np.mean(df.iloc[:,df.columns.str.startswith(f'log_return_{lag}_')], axis=1)
        for id in range(10):
            symbol = list_cryptocurrencies[id] 
            df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
            df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
    # apply TA indicators: EMA, RSI, Bollinger Bands, Stochastic Oscillator
    for id in range(10):
        symbol = list_cryptocurrencies[id]
        for lag in lags_ema:
            # df[f'ema_close_{lag}_{symbol}'] = ta.trend.ema_indicator(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'ema_close_{lag}_{symbol}'] = exponential_moving_average(df[f'close_{symbol}'], span=lag)
        for lag in lags_rsi:
            # df[f'rsi_close_{lag}_{symbol}'] = ta.momentum.rsi(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'rsi_close_{lag}_{symbol}'] = relative_strength_index(df[f'close_{symbol}'], window=lag)
        for lag in lags_bbands:
            # df[f'hbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_hband(df[f'close_{symbol}'], window=lag, fillna=True)
            # df[f'lbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_lband(df[f'close_{symbol}'], window=lag, fillna=True)
            # df[f'width_bbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_wband(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'hbbands_close_{lag}_{symbol}'] = bollinger_hband(df[f'close_{symbol}'], window=lag)
            df[f'lbbands_close_{lag}_{symbol}'] = bollinger_lband(df[f'close_{symbol}'], window=lag)
            df[f'width_bbands_close_{lag}_{symbol}'] = bollinger_wband(df[f'close_{symbol}'], window=lag)
        # Alpha#12: (sign(delta(volume, 1)) * (-1 * delta(close, 1))) 
        df['alpha12_'+symbol] = np.sign(delta(df['volume_'+symbol], 1)) * (-1 * delta(df['close_'+symbol], 1))
        # Alpha#54: ((-1 * ((low - close) * (open^5))) / ((low - high) * (close^5))) 
        df['alpha54_'+symbol] = ((-1 * ((df['low_'+symbol] - df['close_'+symbol]) * (df['open_'+symbol]**5))) / ((df['low_'+symbol] - df['high_'+symbol]) * (df['close_'+symbol]**5)))
        # Alpha#9: ((0 < ts_min(delta(close, 1), 5)) ? delta(close, 1) : ((ts_max(delta(close, 1), 5) < 0) ? delta(close, 1) : (-1 * delta(close, 1)))) 
        df['alpha9_'+symbol] = np.where(0 < ts_min(delta(df['close_'+symbol], 1), 5), delta(df['close_'+symbol], 1), np.where(ts_max(delta(df['close_'+symbol], 1), 5) < 0, delta(df['close_'+symbol], 1), -1 * delta(df['close_'+symbol], 1)))
    gc.collect()
    df.fillna(method='ffill', inplace=True)
    # df.fillna(method='bfill', inplace=True)  
    for symbol in list_cryptocurrencies:
        df.drop(columns=[f'open_{symbol}', f'high_{symbol}', f'low_{symbol}', f'volume_{symbol}'], inplace=True)
    gc.collect()  
    df = df[df['timestamp'] >= limit] 
    df.drop_duplicates(subset=['timestamp'], inplace=True)
    return df   

In [15]:
def getFeaturesVer2(df, limit):
    lags_normal = [8, 20, 120]
    lags_ema = [15]
    lags_rsi = [8, 14]
    lags_stoch_osci = [10, 25]
    roc_window = [8, 15]
    williams_percentage_r = [10, 20]
    lags_wbands = [10, 50]
    for id in range(10):        
        symbol = list_cryptocurrencies[id] 
        for lag in lags_normal:
            df[f'log_close/mean_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.append(np.convolve( np.array(df[f'close_{symbol}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1)  )
            df[f'log_return_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.array(df[f'close_{symbol}']), lag)  )
    for lag in lags_normal:
        df[f'mean_close/mean_{lag}'] =  np.mean(df.iloc[:,df.columns.str.startswith(f'log_close/mean_{lag}_')], axis=1)
        df[f'mean_log_returns_{lag}'] = np.mean(df.iloc[:,df.columns.str.startswith(f'log_return_{lag}_')], axis=1)
        for id in range(10):
            symbol = list_cryptocurrencies[id] 
            df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
            df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
    # apply TA indicators: EMA, RSI, Bollinger Bands, Stochastic Oscillator
    for id in range(10):
        symbol = list_cryptocurrencies[id]
        for lag in lags_ema:
            # df[f'ema_close_{lag}_{symbol}'] = ta.trend.ema_indicator(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'ema_close_{lag}_{symbol}'] = exponential_moving_average(df[f'close_{symbol}'], span=lag)
        for lag in lags_rsi:
            # df[f'rsi_close_{lag}_{symbol}'] = ta.momentum.rsi(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'rsi_close_{lag}_{symbol}'] = relative_strength_index(df[f'close_{symbol}'], window=lag)
        for lag in lags_stoch_osci:
            # df[f'stoch_oscil_close_{lag}_{symbol}'] = ta.momentum.stoch(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'stoch_oscil_close_{lag}_{symbol}'] = stochastic_oscillator(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag)
        for lag in roc_window:
            # df[f'roc_close_{lag}_{symbol}'] = ta.momentum.roc(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'roc_close_{lag}_{symbol}'] = rate_of_change(df[f'close_{symbol}'], window=lag)
        for lag in williams_percentage_r:
            # df[f'williams_close_{lag}_{symbol}'] = ta.momentum.williams_r(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], lbp=lag, fillna=False)  
            df[f'williams_close_{lag}_{symbol}'] = williams_r(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag)
        for lag in lags_wbands:
            # df[f'wbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_wband(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'wbbands_close_{lag}_{symbol}'] = bollinger_wband(df[f'close_{symbol}'], window=lag) 
        # Alpha#12: (sign(delta(volume, 1)) * (-1 * delta(close, 1)))
        df['alpha12_'+symbol] = np.sign(delta(df['volume_'+symbol], 1)) * (-1 * delta(df['close_'+symbol], 1))
        # Alpha#41: (((high * low)^0.5) - close)
        df['alpha41_'+symbol] = ((df['high_'+symbol] * df['low_'+symbol])**0.5) - df['close_'+symbol]
        # Alpha#53: (-1 * delta((((close - low) - (high - close)) / (close - low + 0.0000001)), 9))
        df['alpha53_'+symbol] = -1 * delta((((df['close_'+symbol] - df['low_'+symbol]) - (df['high_'+symbol] - df['close_'+symbol])) / (df['close_'+symbol] - df['low_'+symbol] + 0.0000001)), 9)
        # Alpha#6: (-1 * correlation(open, volume, 10)) 
        df['alpha6_'+symbol] = -1 * correlation(df['open_'+symbol], df['volume_'+symbol], 10)
        # Alpha#9: ((0 < ts_min(delta(close, 1), 5)) ? delta(close, 1) : ((ts_max(delta(close, 1), 5) < 0) ? delta(close, 1) : (-1 * delta(close, 1)))) 
        df['alpha9_'+symbol] = np.where(0 < ts_min(delta(df['close_'+symbol], 1), 5), delta(df['close_'+symbol], 1), np.where(ts_max(delta(df['close_'+symbol], 1), 5) < 0, delta(df['close_'+symbol], 1), -1 * delta(df['close_'+symbol], 1)))
    gc.collect()  
    # df.dropna(inplace=True)
    df.fillna(method='ffill', inplace=True)
    # df.fillna(method='bfill', inplace=True)
    for symbol in list_cryptocurrencies:
        df.drop(columns=[f'high_{symbol}', f'low_{symbol}', f'open_{symbol}', f'volume_{symbol}'], inplace=True)
    gc.collect()  
    df = df[df['timestamp'] >= limit] 
    df.drop_duplicates(subset=['timestamp'], inplace=True)
    return df   

In [16]:
combined_df1 = getFeaturesVer1(combined_df1, lower_limit_test)
print(combined_df1.shape)
combined_df2 = getFeaturesVer2(combined_df2, lower_limit_test)
print(combined_df2.shape)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\403103186.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\403103186.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
C:\

(122699, 419)


C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\3820758813.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\3820758813.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
C

(122699, 307)


C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\3820758813.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['timestamp'], inplace=True)


In [17]:
num_models = 10

In [18]:
def predictForEachAssetVer1(df):
    preds = []
    output = pd.DataFrame()
    for id, symbol in enumerate(list_cryptocurrencies):
        
        pred_asset = []
        symbol = list_cryptocurrencies[id]
        target = f'log_return_{symbol}'
        columns = [col for col in df.columns if symbol in col] + ['timestamp']
        df_asset = df[columns]
        
        #load models
        models = []
        for fold in range(num_models):
            model = pickle.load(open(os.path.join(MODELS_DIR, f'{symbol}_fold{fold}.pkl'), 'rb'))
            models.append(model)
        #predict
        df_asset = df_asset.fillna(method='ffill')
        df_asset = df_asset.drop(columns=[target, 'timestamp']) 
        for model in models: 
            pred_fold = model.predict(df_asset)
            pred_asset.append(pred_fold)
            if symbol == 'XRP':
                break
        
        pred = np.mean(pred_asset, axis=0)
        del models
        preds.append(pred)
        output_symbol = pd.DataFrame({'timestamp': df['timestamp'], 'symbol': symbol, 'log_return': pred})
        output_symbol.fillna(method='ffill', inplace=True)
        output = pd.concat([output, output_symbol])
        gc.collect()
    return output

In [19]:
def predictForEachAssetVer2(df):
    preds = []
    output = pd.DataFrame()
    for id, symbol in enumerate(list_cryptocurrencies):
        
        pred_asset = []
        target = f'log_return_{symbol}'
        columns = [col for col in df.columns if symbol in col] + ['timestamp']
        df_asset = df[columns]
        
        #load models
        models = []
        for fold in range(num_models):
            model = pickle.load(open(os.path.join(MODELS_DIR, f'{symbol}_fold{fold}2.pkl'), 'rb'))
            models.append(model)
        #predict
        df_asset = df_asset.fillna(method='ffill')
        df_asset = df_asset.drop(columns=[target, 'timestamp']) 
        for model in models: 
            pred_fold = model.predict(df_asset)
            pred_asset.append(pred_fold)
            if symbol == 'XRP':
                break
         
        
        pred = np.mean(pred_asset, axis=0)
        del models
        preds.append(pred)
        output_symbol = pd.DataFrame({'timestamp': df['timestamp'], 'symbol': symbol, 'log_return': pred})
        output_symbol.fillna(method='ffill', inplace=True)
        output = pd.concat([output, output_symbol])
        gc.collect()
    return output

In [20]:
output1 = predictForEachAssetVer1(combined_df1)
print(output1.shape)
output2 = predictForEachAssetVer2(combined_df2)
print(output2.shape)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\995792437.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fillna(method='ffill')
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\995792437.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ffill', inplace=True)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\995792437.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fillna(method='ffill')
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\995792437.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ff

(1226990, 3)


C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\1575420756.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ffill', inplace=True)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\1575420756.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fillna(method='ffill')
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\1575420756.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ffill', inplace=True)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\1575420756.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset

(1226990, 3)


C:\Users\e0817820\AppData\Local\Temp\ipykernel_29956\1575420756.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ffill', inplace=True)


In [21]:
def correlation(a, b):    
    a = np.ravel(a)
    b = np.ravel(b)

    len_data = len(a)
    mean_a = np.mean(a)
    mean_b = np.mean(b)
    var_a = np.var(a, ddof=0)  
    var_b = np.var(b, ddof=0)  
    
    cov = np.sum((a - mean_a) * (b - mean_b)) / len_data
    corr = cov / np.sqrt(var_a * var_b)

    return corr

In [22]:
idx = output1[['timestamp', 'symbol']]
log_return1 = output1[['log_return']]
log_return2 = output2[['log_return']]
max_corr = 0
max_k = 0
for k in np.arange(0, 1.05, 0.05):
    print(f"Ver1 coefficient: {k}")
    print(f"Ver2 coefficient: {1-k}")
    log_return_mean = (k*log_return1 + (1-k)*log_return2)
    output = pd.concat([idx, log_return_mean], axis=1)
    output = df_test.merge(output, on=['timestamp', 'symbol'], how='left')[['id', 'log_return', 'timestamp', 'symbol']]
    corr_list = []
    sample = pd.read_csv(r'C:\Users\e0817820\Desktop\tokka\data\raw\test_log_return.csv')
    for symbol in list_cryptocurrencies:
        pred = output[output['symbol'] == symbol]['log_return']
        real = sample[output['symbol'] == symbol]['log_return']
        corr_list.append(correlation(pred, real))
    print(corr_list)
    print(f"Without ensemble: {np.mean(corr_list)}")
    if (np.mean(corr_list) > max_corr):
        max_corr = np.mean(corr_list)
        max_k = k
print(f"Max_corr: {max_corr}, Max_k: {max_k}")

Ver1 coefficient: 0.0
Ver2 coefficient: 1.0
[0.12443255560922817, 0.04300532456353801, 0.038971544117263064, 0.12013194426705051, 0.02473924585855949, 0.06496655467816016, 0.029378582942795838, -0.010792573725838165, 0.018859502432116846, 0.029109852395273948]
Without ensemble: 0.04828025331381479
Ver1 coefficient: 0.05
Ver2 coefficient: 0.95
[0.12421668732043212, 0.042884693787497825, 0.039166048970052283, 0.12062942595327998, 0.02435101367994361, 0.06659019652610516, 0.029028239825371276, -0.011305810304601092, 0.019618422332114912, 0.03003041926726973]
Without ensemble: 0.04852093373574658
Ver1 coefficient: 0.1
Ver2 coefficient: 0.9
[0.12393739200919614, 0.042735773789024496, 0.039349440450453486, 0.12109986791168567, 0.023934955779349273, 0.06816082521190243, 0.028657674381197366, -0.011811592285819853, 0.02040925485085819, 0.030950660645953285]
Without ensemble: 0.04874242527438005
Ver1 coefficient: 0.15000000000000002
Ver2 coefficient: 0.85
[0.12359608513954458, 0.042556188418448

In [23]:
idx = output1[['timestamp', 'symbol']]
k = .7
log_return_mean = (k*log_return1 + (1-k)*log_return2)
output = pd.concat([idx, log_return_mean], axis=1)
output = df_test.merge(output, on=['timestamp', 'symbol'], how='left')[['id', 'log_return', 'timestamp', 'symbol']]
corr_list = []
sample = pd.read_csv(r'C:\Users\e0817820\Desktop\tokka\data\raw\test_log_return.csv')
for id, symbol in enumerate(list_cryptocurrencies):
    pred = output[output['symbol'] == symbol][['log_return']]
    real = sample[output['symbol'] == symbol][['log_return']]
    corr_list.append(correlation(pred, real))
print(corr_list)
print(f"Without ensemble: {np.mean(corr_list)}")
output = output[['id', 'log_return']]
output.to_csv(r'C:\Users\e0817820\Desktop\tokka\data\processed\submission.csv', index=False)

[0.11636755444605022, 0.037887765956192074, 0.039647241076903476, 0.12363500272580134, 0.016356866413762716, 0.08273960447277984, 0.022542124258207452, -0.01624049220257212, 0.032423621733647635, 0.04075719185953721]
Without ensemble: 0.049611648074030984


In [24]:
print(output.shape)

(1226990, 2)
